In [1]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import preprocessing

datas = pd.read_json('../yelp_dataset/yelp_dataset_labeled.json', lines=True).fillna(0)
datas

,business_id,city,label,latitude,longitude,review_count,stars,state,neighbor_count,4star_neighbor_count,active_users_in_city,neighbor_review_count
0,ZrkH2I-ct6rYX1FNBhY1oA,Nahant,1,42.436666,-70.936382,19,3.0,MA,0,0,19,0.0
1,UYWYdJjPjQDTRqhpfGvRLw,Osteen,0,28.843058,-81.164583,5,4.0,FL,0,0,5,0.0
2,odq3dSNenceAyzoyNogosA,South Natick,1,42.263404,-71.309128,12,2.5,MA,0,0,12,0.0
3,uiBZ6yYQ0wv6nwdhkrf4nA,West Vancouver,0,49.334655,-123.180291,2,2.5,BC,13,9,145,62.0
4,vwKdjOBaGprrN3Qyn4u8sw,West Vancouver,0,49.373914,-123.274435,4,3.5,BC,8,3,145,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...
26332,WpBwwZ2qVQmveXd_0CZuJg,Hilliard,0,40.017815,-83.136995,16,4.5,OH,23,9,344,205.0
26333,yHixklhb5guNmcvWkg8qZw,Hilliard,1,40.030409,-83.156736,8,4.0,OH,32,18,344,296.0
26334,GiIWtDUqYHIheAKT0zKs4g,Hilliard,1,40.017995,-83.136033,15,4.0,OH,23,8,344,198.0
26335,i4YPpmtUfkQSCNu1T7aZYA,Berkshire,0,40.267734,-82.924768,2,2.5,OH,0,0,2,0.0


In [2]:
y = datas.iloc[:, 2]
x = datas.drop(columns=['business_id', 'label'])
city = datas['city']
state = datas['state']
city_dummies = pd.get_dummies(city, drop_first=True)
state_dummies = pd.get_dummies(state, drop_first=True)
x = pd.concat(objs=[datas.drop(columns=['business_id', 'label', 'city', 'state']), city_dummies, state_dummies], axis='columns')
x


,city,latitude,longitude,review_count,stars,state,neighbor_count,4star_neighbor_count,active_users_in_city,neighbor_review_count
0,201,42.436666,-70.936382,19,3.0,6,0,0,19,0.0
1,236,28.843058,-81.164583,5,4.0,3,0,0,5,0.0
2,280,42.263404,-71.309128,12,2.5,6,0,0,12,0.0
3,323,49.334655,-123.180291,2,2.5,0,13,9,145,62.0
4,323,49.373914,-123.274435,4,3.5,0,8,3,145,52.0
...,...,...,...,...,...,...,...,...,...,...
26332,136,40.017815,-83.136995,16,4.5,7,23,9,344,205.0
26333,136,40.030409,-83.156736,8,4.0,7,32,18,344,296.0
26334,136,40.017995,-83.136033,15,4.0,7,23,8,344,198.0
26335,27,40.267734,-82.924768,2,2.5,7,0,0,2,0.0


In [3]:
x = sm.add_constant(x) # 若模型中有截距，必须有这一步
model = sm.OLS(y, x).fit() # 构建最小二乘模型并拟合
print(model.summary()) # 输出回归结果

                            OLS Regression Results                            
Dep. Variable:                  label   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     8.748
Date:                Thu, 27 May 2021   Prob (F-statistic):           1.79e-14
Time:                        15:15:07   Log-Likelihood:                -17151.
No. Observations:               26337   AIC:                         3.432e+04
Df Residuals:                   26326   BIC:                         3.441e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.31